In [8]:
import numpy as np
from scipy.optimize import minimize_scalar

In [9]:
def linear_model(x, a, b):
    return a * x + b
def rational_model(x, a, b):
    return a / (1 + b * (x+1e-9))

In [48]:
def loss(f,x, y,a, b):
    return np.sum((f(x, a, b) - y) ** 2)

In [56]:
def my_steep_des(model, x, y, init_a, init_b, eps2=1e-3):
    def check(A1, A2, eps2):
        return np.linalg.norm(A1 - A2) < eps2

    eps = 1e-6
    a1, b1 = init_a, init_b

    # Initial gradient
    d_f_a = (loss(model, x, y, a1 + eps, b1) - loss(model, x, y, a1 - eps, b1)) / (2 * eps)
    d_f_b = (loss(model, x, y, a1, b1 + eps) - loss(model, x, y, a1, b1 - eps)) / (2 * eps)

    # Line search for first step
    res = minimize_scalar(lambda alpha: loss(model, x, y, a1 - alpha * d_f_a, b1 - alpha * d_f_b),
                          bounds=(0, 1), method='bounded')
    first_beta = res.x
    a2 = a1 - first_beta * d_f_a
    b2 = b1 - first_beta * d_f_b

    while True:
        # Gradients at a1,b1 and a2,b2
        d_f_a1 = (loss(model, x, y, a1 + eps, b1) - loss(model, x, y, a1 - eps, b1)) / (2 * eps)
        d_f_b1 = (loss(model, x, y, a1, b1 + eps) - loss(model, x, y, a1, b1 - eps)) / (2 * eps)
        d_f_a2 = (loss(model, x, y, a2 + eps, b2) - loss(model, x, y, a2 - eps, b2)) / (2 * eps)
        d_f_b2 = (loss(model, x, y, a2, b2 + eps) - loss(model, x, y, a2, b2 - eps)) / (2 * eps)

        df1 = np.array([d_f_a1, d_f_b1])
        df2 = np.array([d_f_a2, d_f_b2])
        AB1 = np.array([a1, b1])
        AB2 = np.array([a2, b2])

        S = AB2 - AB1
        DF = df2 - df1
        beta = abs(np.dot(S, DF)) / np.dot(DF, DF)
        AB_next = AB2 - beta * df2


        if check(AB2, AB_next, eps2):
            break

        a1, b1 = a2, b2
        a2, b2 = AB_next[0], AB_next[1]



    return a2, b2,loss(model, x, y, a2 , b2)


In [64]:
def gra_des(model,x,y,init_a,init_b,eps2=1e-3):
    def check(A1, A2, eps2):
        return np.linalg.norm(A1 - A2) < eps2

    eps = 1e-6
    a0, b0 = init_a, init_b

    # Initial gradient
    dfa0 = (loss(model, x, y, a0 + eps, b0) - loss(model, x, y, a0 - eps, b0)) / (2 * eps)
    dfb0 = (loss(model, x, y, a0, b0 + eps) - loss(model, x, y, a0, b0 - eps)) / (2 * eps)


    res = minimize_scalar(lambda alpha: loss(model, x, y, a0 - alpha * dfa0, b0 - alpha * dfb0),
                          bounds=(0, 1), method='bounded')
    beta0 = res.x
    a1 = a0 - beta0 * dfa0
    b1 = b0 - beta0 * dfa0

    while True:

        dfa_new = -(loss(model, x, y, a1 + eps, b1) - loss(model, x, y, a1 - eps, b1)) / (2 * eps)
        dfb_new = -(loss(model, x, y, a1, b1 + eps) - loss(model, x, y, a1, b1 - eps)) / (2 * eps)
        DF_new= np.array([dfa_new, dfb_new])
        DF_old = np.array([-dfa0, -dfb0])
        s0=DF_old
        beta_FR = (np.dot(DF_new, DF_new))/(np.dot(DF_old, DF_old))
        beta_PR= (np.dot(DF_new, DF_new-DF_old))/(np.dot(DF_old, DF_old))
        beta_FR_new=DF_new+beta_FR*s0
        beta_PR_new=DF_new+beta_PR*s0
        s_new_FR= DF_new+ beta_FR*s0
        res = minimize_scalar(lambda alpha: loss(model, x, y, a1 + alpha *DF_new, b1 + alpha * DF_new),
                          bounds=(0, 1), method='bounded')

        a2, b2 = a1+res.x*s_new_FR , b1+res.x*s_new_FR


        df1 = np.array([d_f_a1, d_f_b1])
        df2 = np.array([d_f_a2, d_f_b2])

        S0 =-df1


        S1=df2+beta_FR*S0

        res = minimize_scalar(lambda alpha: loss(model, x, y, a2 +alpha*S1[0], b2 - alpha * S1[1]),
                          bounds=(0, 1), method='bounded')

        AB2 = np.array([a2, b2])
        AB_next = AB2 + res.x * S1

        if check(AB2, AB_next, eps2):
            break

        a1, b1 = a2, b2
        a2, b2 = AB_next[0], AB_next[1]


    return a2, b2,loss(model, x, y, a2 , b2)



In [65]:
np.random.seed(23)
alpha = np.random.rand()
beta = np.random.rand()

k = np.arange(101)
x = k / 100
delta = np.random.normal(0, 1, size=len(x))
y = alpha * x + beta + delta*0

print (alpha)
print (beta)


0.5172978838465893
0.9469626038148141


In [66]:
a,b,m=my_steep_des(linear_model,x,y,1, 1)
print (a,b,m)

0.5173202009773403 0.9470041489652785 2.84821644376747e-07


In [67]:
a,b,m=gra_des(linear_model,x,y,1, 1)
print (a,b,m)

1.231634365959868e+20 2.277793801542807e+20 8.58694223274801e+42
